In [7]:
%matplotlib inline  
import matplotlib.pyplot as plt
from pamtra2.libs.singleScattering import scattering
from pamtra2.libs.singleScattering import scattering_utilities as scautils
from pamtra2.libs.singleScattering import mie, rayleigh, tmatrix, ssrg
from pamtra2.libs import refractiveIndex
import numpy as np
np.set_printoptions(precision=3)
rad2deg = 180.0/np.pi

# Angles
t0 = np.pi*0.75 # theta must be within 0 pi
t1 = np.pi*0.25
#t1 = np.pi-t0
p0 = 0.4*np.pi
p1 = 0.8*np.pi
p1 = np.pi+p0
print(p1, p1 // (2.*np.pi))
#p1 = p1 - 2.*np.pi*(p1//(2.*np.pi))
print(t0, t1, p0, p1)

4.39822971502571 0.0
2.356194490192345 0.7853981633974483 1.2566370614359172 4.39822971502571


In [8]:
# One centimeter particle, S-band 2.8GHz, pure ice sphere 
f = 2.8e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')

cost=1.0
R = rayleigh.RayleighScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
M = mie.MieScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
T = tmatrix.TmatrixScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
S = ssrg.SsrgScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1, volume=np.pi*d**3/6.)

from pytmatrix import tmatrix as tm
from pytmatrix import scatter, radar
scatt = tm.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(R.Cext, R.Csca, R.Cabs, R.Cbck)
print(M.Cext, M.Csca, M.Cabs, M.Cbck)
print(S.Cext, S.Csca, S.Cabs, S.Cbck)
print(T.Cext, T.Csca, T.Cabs, T.Cbck)


print(" ")
print("RAY")
print(" ")
print(R.S)
print(" ")
#print(R.S2/(R.S1*np.cos(p1-p0)))
print(" ")
print("MIE")
print(M.S)
print(" ")
#print(M.S2/(M.S1*np.cos(p1-p0)))
print(" ")
print("SSRG")
print(" ")
print(S.S)
print(" ")
print("TMM")
print(T.S)
print(" ")
#print(T.S[1,1]/(T.S[0,0]*np.cos(p1-p0)))
print(" ")
print(scatt.get_S())
print(" ")
print(T.S)
print(" ")
print(" ")


Cext                    Csca                    Cabs                   Cbck            [m**2]
3.42313882614e-12 2.74872213037e-13 3.1482666131e-12 4.12308319556e-13
3.42552492696e-12 2.74936863749e-13 3.15058806321e-12 4.12217233995e-13
3.42307506305e-12 2.748084499522858e-13 3.1482666131e-12 4.12117043789e-13
3.42552500089e-12 2.7493685025226993e-13 3.15058815064e-12 4.12217233682e-13
 
RAY
 
[[  1.811e-07 +1.470e-11j   0.000e+00 +0.000e+00j]
 [  0.000e+00 +0.000e+00j  -1.811e-07 -1.470e-11j]]
 
 
MIE
[[  1.811e-07 +1.599e-11j   0.000e+00 +0.000e+00j]
 [  0.000e+00 +0.000e+00j  -1.811e-07 -1.599e-11j]]
 
 
SSRG
 
[[  1.811e-07 +1.470e-11j   0.000e+00 +0.000e+00j]
 [  0.000e+00 +0.000e+00j  -1.811e-07 -1.470e-11j]]
 
TMM
[[  1.811e-07 +1.599e-11j   2.561e-14 +2.261e-18j]
 [  2.561e-14 +2.261e-18j  -1.811e-07 -1.599e-11j]]
 
 
[[  1.811e-07 +1.599e-11j   2.561e-14 +2.261e-18j]
 [  2.561e-14 +2.261e-18j  -1.811e-07 -1.599e-11j]]
 
[[  1.811e-07 +1.599e-11j   2.561e-14 +2.261e-18j]
 [  2.

In [9]:
TS = scautils.amplitude_matrix(T.S)
TZ = scautils.amplitude2mueller(TS)
MS = scautils.amplitude_matrix(M.S)
MZ = scautils.amplitude2mueller(MS)
print(TZ.matrix)
print("")
print(MZ.matrix)
print("")
print(scatt.get_Z())

[[  3.280e-14   3.566e-28  -4.965e-35  -3.673e-40]
 [  3.566e-28   3.280e-14  -9.278e-21   1.837e-40]
 [ -4.965e-35  -9.278e-21  -3.280e-14   7.704e-34]
 [  3.673e-40  -1.837e-40  -7.704e-34  -3.280e-14]]

[[  3.280e-14   0.000e+00  -0.000e+00  -0.000e+00]
 [  0.000e+00   3.280e-14  -0.000e+00  -0.000e+00]
 [ -0.000e+00  -0.000e+00  -3.280e-14   0.000e+00]
 [ -0.000e+00  -0.000e+00   0.000e+00  -3.280e-14]]

[[  3.280e-14   3.566e-28  -4.965e-35  -3.673e-40]
 [  3.566e-28   3.280e-14  -9.278e-21   1.837e-40]
 [ -4.965e-35  -9.278e-21  -3.280e-14   7.704e-34]
 [  3.673e-40  -1.837e-40  -7.704e-34  -3.280e-14]]


In [ ]:
# One millimeter particle, W-band 94GHz, pure ice sphere 
f = 94e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')
rayC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='TMM', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)
from pytmatrix import tmatrix, scatter, radar
scatt = tmatrix.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)
print([scatter.ext_xsect(scatt),scatter.sca_xsect(scatt),scatter.ext_xsect(scatt)-scatter.sca_xsect(scatt),radar.radar_xsect(scatt)])

In [ ]:
# One millimeter particle, S-band 2.8GHz, pure ice sphere 
f=2.8e9
n = refractiveIndex.n(frequencies=f,temperatures=263.15,substance='ice')
rayC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='TMM')

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)

In [ ]:
scatt.get_S()
scatt.get_Z()